# Page for trying out the data

In [1]:
import pandas as pd
import time, os, shutil
import nltk
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np  
from sklearn.cluster import KMeans  
from sklearn.cluster import DBSCAN


W0502 15:54:05.637127  5876 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [11]:
#df1 = pd.read_csv('amazon_reviews_us_Mobile_Electronics_v1_00.tsv',sep="\t", error_bad_lines=False)
# bad lines exist......
#data_source = 'amazon_reviews_us_Mobile_Electronics_v1_00.tsv'
data_source='amazon_reviews_us_PC_v1_00.tsv.gz'
#data_source='amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'
df1 = pd.read_csv(data_source,sep="\t", error_bad_lines=False)

b'Skipping line 42306: expected 15 fields, saw 22\nSkipping line 61136: expected 15 fields, saw 22\nSkipping line 64592: expected 15 fields, saw 22\n'
b'Skipping line 73268: expected 15 fields, saw 22\nSkipping line 80720: expected 15 fields, saw 22\nSkipping line 120624: expected 15 fields, saw 22\nSkipping line 121776: expected 15 fields, saw 22\n'
b'Skipping line 154351: expected 15 fields, saw 22\nSkipping line 155937: expected 15 fields, saw 22\n'
b'Skipping line 203615: expected 15 fields, saw 22\nSkipping line 208061: expected 15 fields, saw 22\nSkipping line 212638: expected 15 fields, saw 22\nSkipping line 219390: expected 15 fields, saw 22\nSkipping line 235195: expected 15 fields, saw 22\n'
b'Skipping line 284570: expected 15 fields, saw 22\nSkipping line 297246: expected 15 fields, saw 22\nSkipping line 326901: expected 15 fields, saw 22\n'
b'Skipping line 344609: expected 15 fields, saw 22\nSkipping line 347706: expected 15 fields, saw 22\nSkipping line 348978: expected 15

b'Skipping line 4382584: expected 15 fields, saw 22\n'
b'Skipping line 4794835: expected 15 fields, saw 22\nSkipping line 4840336: expected 15 fields, saw 22\n'
b'Skipping line 5002329: expected 15 fields, saw 22\nSkipping line 5012210: expected 15 fields, saw 22\nSkipping line 5021546: expected 15 fields, saw 22\n'
b'Skipping line 5117714: expected 15 fields, saw 22\n'
b'Skipping line 5382553: expected 15 fields, saw 22\nSkipping line 5405521: expected 15 fields, saw 22\n'
b'Skipping line 5652268: expected 15 fields, saw 22\n'
b'Skipping line 5761025: expected 15 fields, saw 22\n'
b'Skipping line 6094502: expected 15 fields, saw 22\n'
b'Skipping line 6875074: expected 15 fields, saw 22\n'


### DATA COLUMNS:
marketplace       - 2 letter country code of the marketplace where the review was written.  
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.  
review_id         - The unique ID of the review.  
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.  
product_parent    - Random identifier that can be used to aggregate reviews for the same product.  
product_title     - Title of the product.  
product_category  - Broad product category that can be used to group reviews  
                    (also used to group the dataset into coherent parts).  
star_rating       - The 1-5 star rating of the review.  
helpful_votes     - Number of helpful votes.  
total_votes       - Number of total votes the review received.  
vine              - Review was written as part of the Vine program.  
verified_purchase - The review is on a verified purchase.  
review_headline   - The title of the review.  
review_body       - The review text.  
review_date       - The date the review was written.  

Filter out the ones not versified and get a subset of the data

In [12]:
df1 = df1.loc[df1['verified_purchase']=='Y',['review_id', 'product_id', 'product_title', 'helpful_votes','review_headline', 'review_body']]
df1['product_id'].nunique()

404509

Get a list of products that are with at least 200 reviews

In [13]:
count_df = df1.groupby('product_id').count()
count_df = count_df['review_id']

count_df.describe()

count    404509.000000
mean         14.949848
std         120.816383
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max       20216.000000
Name: review_id, dtype: float64

In [ ]:
count_df = count_df.loc[lambda x: x>=200]
count_df = count_df.sort_values(ascending=False)

product_list = count_df.index.values.tolist()

In [5]:
df1 = df1[df1['product_id'].isin(product_list)]

df1['product_title'][0:4].tolist()

['iXCC Multi pack Lightning cable',
 'Generic Car Dashboard Video Camera Vehicle Video Accident Recorder (2.0" 1080P)',
 'iXCC Element II Lightning Cable 6ft, iPhone Charger, for iPhone 7 6s 6 Plus, SE 5s 5c 5, iPad Air 2 Pro, iPad mini 2 3 4, iPad 4th Gen [Apple MFi Certified](White and Black)',
 'iXCC Element II Lightning Cable 6ft, iPhone Charger, for iPhone 7 6s 6 Plus, SE 5s 5c 5, iPad Air 2 Pro, iPad mini 2 3 4, iPad 4th Gen [Apple MFi Certified](White and Black)']

Test on one product. Get the reviews for one product

In [6]:
reviews = df1[df1['product_id']==product_list[1]]#.drop(columns=['review_id','product_title','product_id'])
#title_='Yamaha YPG-235 76-Key Portable Grand Piano Premium Pack'
#reviews = df1[df1['product_title']==title_]
product_title = reviews['product_title'].tolist()[0]
product_title

'Wall AC Charger USB Sync Data Cable for iPhone 4, 3GS, and iPod'

In [ ]:
reviews = reviews['review_body']


reviews = reviews.str.replace('<br />','')

def remove_consecutive(text):
    one = re.sub(r"([eoEO])\1\1+",r"\1\1",text)
    return re.sub(r"([^eoEO])\1\1+",r"\1",one)

reviews = pd.Series(list(map(remove_consecutive,reviews)))

In [ ]:
reviews = reviews.str.split("[.!?]+",expand=True).stack().reset_index() # split sentences
#remove empty lines
reviews['word_count'] = reviews[0].str.split().apply(len)#.strip().apply(len)
reviews = reviews.loc[reviews['word_count']>4,[0]]

reviews['text'] = reviews[0].str.strip()
reviews = reviews.drop(columns=0)

In [ ]:
#reviews.to_csv(r'/mnt/doc/School/USYD/Capstone/workspace/data_testing/output/sub-set/AmazonBasics Wireless Mouse.csv',sep='\t')

In [ ]:
# Part of speech tagging
reviews['POS'] = reviews['text'].apply(nltk.word_tokenize).apply(nltk.pos_tag)

def getMainWords(pos):
    result = '';
    for x in pos:
        if x[1][0:2] in ['NN','VB']:
            result= result+x[0]+' '
    if len(result)>0:
        result= result[0:len(result)-1]
    return result

reviews['main'] = list(map(getMainWords,reviews['POS']))

In [ ]:
reviews = reviews.loc[reviews['main'].apply(len)>0,['text','main']] #filtering out sentences without any nouns or verbs

reviews = reviews.reset_index(drop=True)# reset index

In [ ]:
reviews

Word Embedding / Sentence Embedding

In [ ]:
# Load google Universal Sentence Encoder
module_dir ="downloads/encoder-DNA" #"downloads/encoder"
embed = hub.Module(module_dir)

In [ ]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    start_time=time.time()
    reviews['sentence_embedding'] = pd.Series(list(session.run(embed(list(reviews['text'])))))
    end_time1=time.time()
    reviews['words_embedding'] = pd.Series(list(session.run(embed(list(reviews['main'])))))
    end_time2=time.time()
    print('time1:',(end_time1-start_time),' time2:',(end_time2-end_time1))

In [ ]:
reviews

In [ ]:
# Preparing datasets
X1 = np.array(reviews['sentence_embedding'].tolist())
X2 = np.array(reviews['words_embedding'].tolist())

In [ ]:
# Clustering
#km_s = KMeans(n_clusters=10).fit(X1)  

#km_w = KMeans(n_clusters=10).fit(X2)

db_s = DBSCAN(eps=0.7).fit(X1)

db_w = DBSCAN(eps=0.7).fit(X2)

In [ ]:
# Get labels
#reviews['label_km_sentence'] = km_s.labels_.tolist()
#reviews['label_km_word'] = km_w.labels_.tolist()

labels_s = db_s.labels_.tolist()
labels_w = db_w.labels_.tolist()
reviews['label_db_sentence'] = labels_s
reviews['label_db_word'] = labels_w


In [ ]:
len(set(labels_s)) - (1 if -1 in labels_s else 0)

In [ ]:
len(set(labels_w)) - (1 if -1 in labels_w else 0)

In [ ]:
def save_results_dir(product_name, method, params):
    dir_pre = '/home/betty35/桌面/Capstone/workspace/data_testing/output/'
    dir_ = dir_pre+product_name+'/'+method+'/'+params+'/'
    return dir_


def make_dir(dir_):
    if not os.path.exists(dir_):
        try:
            os.makedirs(dir_)
        except FileExistsError:
            pass

def clean_dir(dir_):
    for file_ in os.listdir(dir_):
        file_dir = os.path.join(dir_, file_dir)
        try:
            if os.path.isfile(file_dir):
                os.unlink(file_dir)
            elif os.path.isdir(file_dir): shutil.rmtree(file_dir)
        except Exception as e:
            print(e)

def make_or_clean_dir(dir_):
    make_dir(dir_)
    clean_dir(dir_)

In [ ]:
n_clusters_s = len(set(labels_s)) - (1 if -1 in labels_s else 0)
n_clusters_w = len(set(labels_w)) - (1 if -1 in labels_w else 0)


for i in range(0,n_clusters_s):
    dir1 = save_results_dir(product_title,'dbscan','0.7/sentence')
    make_or_clean_dir(dir1)
    temp = reviews.loc[reviews['label_db_sentence']==i,['text']]
    file_name = str(i)+'.txt'
    np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')


#for i in range(0,n_clusters_w):
 #   dir1 = '/home/betty35/桌面/Capstone/workspace/data_testing/output/dbscan/'
  #  temp = reviews.loc[reviews['label_db_word']==i,['text']]
   # file_name = 'word/'+str(i)+'.txt'
    #np.savetxt(r''+dir1+file_name, temp.values, fmt='%s')

In [ ]:
# Save results for km

for i in range(0,9):
    temp = reviews.loc[reviews['label_km_sentence']==i,['text']]
    temp2 = reviews.loc[reviews['label_km_word']==i,['text']]
    file_name = 'km10/sentence/'+str(i)+'.txt'
    file_name2 = 'km10/word/'+str(i)+'.txt'
    np.savetxt(r'/home/betty35/桌面/Capstone/workspace/data_testing/output/'+file_name, temp.values, fmt='%s')
    np.savetxt(r'/home/betty35/桌面/Capstone/workspace/data_testing/output/'+file_name2, temp2.values, fmt='%s')